In [3]:
import praw
import pandas as pd
import numpy as np

# Reddit Connection

In [4]:
info = pd.read_csv('../best-comment/connection.csv')
reddit = praw.Reddit(
    client_id=info['client_id'][0],
    client_secret=info['client_secret'][0],
    user_agent=info['user_agent'][0],
    username=info['username'][0],
    password=info['password'][0]
)

In [39]:
data = pd.read_csv('seeker_satisfaction_1000_thread_containing_content.csv')
len(data['post_id'])

1000

# 중복 post 확인

In [40]:
duplicated = []
dupes = data['post_id'].duplicated()
length = len(dupes)
for i in range(length):
    if dupes[i]:
        duplicated.append(data['post_id'][i])

In [41]:
len(data[data['post_id'].isin(duplicated)])

12

# Contents 가져오기

In [71]:
post_titles = []
post_contents = []
comment_bodys = []

for post_id, comment_id in zip(list(data['post_id']), list(data['comment_id'])):
    post_titles.append(reddit.submission(id=post_id).title)
    post_contents.append(reddit.submission(id=post_id).selftext)
    comment_bodys.append(reddit.comment(id=comment_id).body)

In [86]:
data_copy = data.copy()
data_copy.insert(3, 'post_title', post_titles)
data_copy.insert(4, 'post_content', post_contents)
data_copy.insert(10, 'comment_body', comment_bodys)

In [93]:
data_copy.to_csv('seeker_satisfaction_1000_thread_containing_content.csv', index=False)

# 하나의 submission에 달린 모든 comment_key 가져오기

In [43]:
submission = reddit.submission(id=data['post_id'][0])
# comments.list() 사용 시, 모든 comment_key 확인 가능.
len(submission.comments.list())

2

# post, comment, reply에서 가져올 수 있는 모든 정보 가져오기

In [100]:
data = pd.read_csv('seeker_satisfaction_1000_thread_containing_content.csv')

In [130]:
post_utc = []
post_num_comments = []
post_score = []
post_upvote_ratio = []
comment_score = []
reply_utc = []

i = 1
for post_id, comment_id, reply_id in zip(list(data['post_id']), list(data['comment_id']), list(data['reply_id'])):
    # about post.
    post_utc.append(reddit.submission(id=post_id).created_utc)
    post_num_comments.append(reddit.submission(id=post_id).num_comments)
    post_score.append(reddit.submission(id=post_id).score)
    post_upvote_ratio.append(reddit.submission(id=post_id).upvote_ratio)
    
    # about comment.
    comment_score.append(reddit.comment(id=comment_id).score)
    
    # about reply.
    reply_utc.append(reddit.comment(id=reply_id).created_utc)
    
    # 진행도를 알기 위해 print i ...
    print(i, end =' ')
    i += 1

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [133]:
post_between_comment = []
comment_between_reply = []

for post, comment, reply in zip(post_utc, data['comment_utc'], reply_utc):
    post_between_comment.append(comment - post)
    comment_between_reply.append(reply - comment)

In [143]:
for i in range(1000):
    post_utc[i] = int(post_utc[i])
    reply_utc[i] = int(reply_utc[i])
    post_between_comment[i] = int(post_between_comment[i])
    comment_between_reply[i] = int(comment_between_reply[i])

In [144]:
data_copy = data.copy()

In [154]:
# data_copy.insert(4, 'post_utc', post_utc)
# data_copy.insert(5, 'post_num_comments', post_num_comments)
# data_copy.insert(6, 'post_score', post_score)
# data_copy.insert(7, 'post_upvote_ratio', post_upvote_ratio)
# data_copy.insert(16, 'comment_score', comment_score)
# data_copy.insert(21, 'reply_utc', reply_utc)
data_copy.insert(25, 'post_between_comment', post_between_comment)
data_copy.insert(26, 'comment_between_reply', comment_between_reply)

In [156]:
data_copy.to_csv('seeker_satisfaction_1000_thread_containing_content_update.csv', index=False)

# comment support score와 post seek score 차이 구하기

In [7]:
data = pd.read_csv('seeker_satisfaction_1000_thread_containing_content_update.csv')

In [12]:
post_is = list(data['post_IS'])
post_es = list(data['post_ES'])
comment_is = list(data['comment_IS'])
comment_es = list(data['comment_ES'])

data_len = len(post_is)
is_difference = []
es_difference = []

for i in range(data_len):
    is_difference.append(comment_is[i] - post_is[i])
    es_difference.append(comment_es[i] - post_es[i])

In [22]:
difference_sum = []
for i in range(data_len): 
    difference_sum.append(is_difference[i] + es_difference[i])

In [25]:
data_copy = data.copy()
data_copy.insert(27, 'IS_difference', is_difference)
data_copy.insert(28, 'ES_difference', es_difference)
data_copy.insert(29, 'difference_sum', difference_sum)

In [27]:
data_copy.to_csv('seeker_satisfaction_1000_thread_containing_content_update2.csv', index=False)